In [1]:
using Revise

In [2]:
using Flux
using JLD2
using FileIO
using MLDataPattern
using CoordinateTransformations
using ProgressMeter
using RigidBodyDynamics
using Gurobi
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
import FluxExtensions
import LearningMPC
import LCPSim
import Hoppers

In [6]:
samples = load("2018-01-30-hopper-grid/grid_search.jld2")["samples"];

In [7]:
maximum(samples) do sample
    sample.state[2]
end

1.0

In [10]:
filter!(samples) do sample
    sample.state[3] <= sample.state[1]
end

54120-element Array{LearningMPC.Sample{Float64},1}:
 LearningMPC.Sample{Float64}([0.25, 0.25, -5.0, -5.0], [-0.0 -3.44884e-13 … 7.031e-15 -1.62567e-21; 40.0 -5.19332e-13 … 8.27324e-15 9.80654e-27], [445.908], LearningMPC.MIPResults
  solvetime_s: Float64 0.0416629
  objective_value: Float64 232.332
  objective_bound: Float64 210.503
)   
 LearningMPC.Sample{Float64}([0.5, 0.25, -5.0, -5.0], [-0.0 -1.41031e-14 … 8.60962e-16 -1.26337e-21; 16.6017 -345.877 … -19.851 -1.27833e-7], [523.856], LearningMPC.MIPResults
  solvetime_s: Float64 0.0379159
  objective_value: Float64 262.657
  objective_bound: Float64 262.657
)       
 LearningMPC.Sample{Float64}([0.75, 0.25, -5.0, -5.0], [-0.0 -1.68556e-13 … -1.29977e-14 1.41063e-28; 40.0 1.45134e-13 … -1.59506e-14 -4.93574e-21], [560.528], LearningMPC.MIPResults
  solvetime_s: Float64 0.0867951
  objective_value: Float64 286.853
  objective_bound: Float64 286.853
)
 LearningMPC.Sample{Float64}([1.0, 0.25, -5.0, -5.0], [-0.0 -3.61778e-12 … 3.98187e-

In [16]:
minimum(samples) do sample
    sample.mip.objective_bound
end

6.934225499940567

In [17]:
maximum(samples) do sample
    sample.mip.objective_value
end

843.608036279683

In [19]:
robot = Hoppers.Hopper()
xstar = Hoppers.nominal_state(robot)
ustar = zeros(num_velocities(xstar))
basevis = Visualizer()[:hopper]
setgeometry!(basevis, robot)
settransform!(basevis[:robot], xstar)

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")
Δt = 0.05
Jc = LCPSim.ContactLQR.contact_jacobian(xstar, [Point3D(default_frame(foot), 0., 0., 0.)])
A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)

([0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [0.0 0.0; 0.0 0.0; 0.0 1.0; 0.0 1.0], [0.0, 0.0, -9.81, -9.81])

In [20]:
features(sample::LearningMPC.Sample) = (sample.state, sample.mip.objective_bound, sample.mip.objective_value, sample.uJ[:, 1])
data = features.(samples);
train_data, test_data = splitobs(shuffleobs(data), at=0.85);

In [28]:
function setup_model(R, B)
    model = FluxExtensions.TangentPropagator(Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 32, elu),
        Dense(32, 32, elu),
        Dense(32, 1, elu),
        AffineMap(fill(900, 1, 1), [450])
        ))
    RiBt = inv(R) * B' 

    function sample_loss(w_tangent)
        (x, lb, ub, input) -> begin
            y, J = model(x)
            value_cost = sum(ifelse.(y .< lb, lb .- y, ifelse.(y .> ub, y .- ub, 0 .* y)))
            tangent_cost = w_tangent * sum(abs2.(-RiBt * J' .- input))
            value_cost + tangent_cost
        end
    end
    function batch_loss(w_tangent)
        loss = sample_loss(w_tangent)
        (samples) -> sum((sample -> loss(sample...)).(samples)) / length(samples)
    end
    model, batch_loss
end

model, loss = setup_model(R, B)
opt = Flux.ADADelta(params(model))

(::#71) (generic function with 1 method)

In [29]:
loss(0.0)(train_data[1:1000])

Tracked 0-dimensional Array{Float64,0}:
214.721

In [30]:
model([1., 1., 0., 0.])

(param([531.55]), param([28.1471 50.9719 93.7818 23.2117]))

In [ ]:
train_loss = loss(0.0)
@showprogress for i in 1:100
    batches = eachbatch(shuffleobs(train_data), size=50)
    for batch in batches
        l = train_loss(batch)
        isinf(Flux.Tracker.value(l)) && error("Loss is Inf")
        isnan(Flux.Tracker.value(l)) && error("Loss is NaN")
        Flux.back!(l)
        opt()
    end
    @show loss(1.0)(train_data) loss(0.0)(train_data)
end

In [40]:
jldopen("2018-01-25-value-tangents-data.jld2", "w") do file
    file["model"] = model
    file["params"] = params(model)
end

6-element Array{Any,1}:
 param([-0.874318 -0.894906 0.449782 -0.29218; -0.0491876 -0.916839 0.0816046 -0.0822522; … ; -0.753808 -2.2484 0.0164334 -0.0220471; -0.135119 -0.0400157 0.206078 -0.206259])                                              
 param([-0.685669, -0.508003, 0.0230763, 0.150286, -0.280697, -0.0709213, 0.370353, 0.528606, -0.17281, -1.45689  …  -0.12054, 0.0365872, 0.5142, 0.171768, -1.13385, 0.0225197, -1.77688, -0.431941, -0.724202, -0.0487545])
 param([0.0827438 0.181199 … 0.0714366 0.429206; 0.296025 0.318421 … 0.190893 0.398564; … ; -0.350043 -0.300057 … -0.0530053 -0.973669; 0.331874 0.488173 … 0.620462 0.636777])                                              
 param([-0.876633, -0.984719, -0.130419, -1.70156, 1.24424, -0.45569, 0.0369737, -1.02634, -1.16589, -0.681811  …  -0.480377, -1.24427, -1.37558, -0.426286, -1.05001, -0.729481, -0.587845, 2.35657, 1.83911, -0.917382])   
 param([0.0186335 0.091613 … 0.00885737 0.00890532])                                    

In [45]:
import LCPSim
import Hoppers
using RigidBodyDynamics
using Gurobi
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

Process(`/home/rdeits/locomotion/explorations/learning-mpc/packages/v0.6/DrakeVisualizer/src/../deps/usr/bin/drake-visualizer`, ProcessRunning)

In [46]:
robot = Hoppers.Hopper()
xstar = Hoppers.nominal_state(robot)
ustar = zeros(num_velocities(xstar))
basevis = Visualizer()[:hopper]
setgeometry!(basevis, robot)
settransform!(basevis[:robot], xstar)

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")
Δt = 0.05
Jc = LCPSim.ContactLQR.contact_jacobian(xstar, [Point3D(default_frame(foot), 0., 0., 0.)])
A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)

([0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [0.0 0.0; 0.0 0.0; 0.0 1.0; 0.0 1.0], [0.0, 0.0, -9.81, -9.81])

In [47]:
model(zeros(4))

(param([223.087]), param([-32.3593 3.97868 -0.706242 0.638184]))

In [48]:
net_value_controller = state -> begin
    x = state_vector(state)
    value, jac = model(x)
    u = vec(-inv(R) * B' * Flux.Tracker.value(jac)')
end

(::#99) (generic function with 1 method)

In [51]:
x_init = MechanismState{Float64}(robot.mechanism)
set_configuration!(x_init, [1.0, 1.0])
set_velocity!(x_init, [0., 0.])
LearningMPC.randomize!(x_init, x_init, 0.5, 1.0)
results = LCPSim.simulate(x_init, net_value_controller,
    robot.environment,
    Δt,
    100,
    GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [52]:
LearningMPC.playback(basevis[:robot], results, Δt)